Download the Dataset

In [ ]:
import os
import zipfile

# Setting up Kaggle API
kaggle_json_path = os.path.expanduser("~/.kaggle/kaggle.json")  # Linux/Mac 
if os.name == 'nt':  
    kaggle_json_path = os.path.expanduser(r"~\.kaggle\kaggle.json")  # Windows 

if not os.path.exists(kaggle_json_path):
    raise FileNotFoundError("Please make sure 'kaggle.json' in the correct dir : {}".format(kaggle_json_path))

# Use Kaggle API download data
competition_name = "fa-24-tamu-csce-633-600-machine-learning"
download_command = f"kaggle competitions download -c {competition_name}"
os.system(download_command)

# unzip data
zip_filename = f"{competition_name}.zip"
if os.path.exists(zip_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall() 
else:
    print(f"Unable to find the file: {zip_filename}")

# List extracted files
extracted_files = os.listdir()


数据已解压: fa-24-tamu-csce-633-600-machine-learning.zip
解压后的文件: ['cgm_test.csv', 'cgm_train.csv', 'demo_viome_test.csv', 'demo_viome_train.csv', 'fa-24-tamu-csce-633-600-machine-learning.zip', 'Final_Project.ipynb', 'img_test.csv', 'img_train.csv', 'label_test_breakfast_only.csv', 'label_train.csv']


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Step 1: 加载数据
cgm_train = pd.read_csv('cgm_train.csv')
cgm_test = pd.read_csv('cgm_test.csv')
demo_train = pd.read_csv('demo_viome_train.csv')
demo_test = pd.read_csv('demo_viome_test.csv')
img_train = pd.read_csv('img_train.csv')
img_test = pd.read_csv('img_test.csv')
label_train = pd.read_csv('label_train.csv')
label_test = pd.read_csv('label_test_breakfast_only.csv')

# Step 2: 日期列处理
def clean_datetime_columns(df, columns):
    for col in columns:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce').astype('int64') // 10**9  # 转为时间戳
        except Exception as e:
            print(f"无法转换列 {col} 为时间戳：{e}")
            df[col] = np.nan  # 填充为缺失值

# 确认需要处理的时间列
cgm_datetime_cols = ['Breakfast Time', 'Lunch Time']  # 修改为实际的时间列名
clean_datetime_columns(cgm_train, cgm_datetime_cols)
clean_datetime_columns(cgm_test, cgm_datetime_cols)

# Step 3: 填充缺失值
def process_dataframe(df):
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
    return numeric_columns, non_numeric_columns

cgm_numeric_cols, cgm_non_numeric_cols = process_dataframe(cgm_train)
demo_numeric_cols, demo_non_numeric_cols = process_dataframe(demo_train)

# 数值型列填充为均值
num_imputer = SimpleImputer(strategy='mean')
cgm_train[cgm_numeric_cols] = num_imputer.fit_transform(cgm_train[cgm_numeric_cols])
cgm_test[cgm_numeric_cols] = num_imputer.transform(cgm_test[cgm_numeric_cols])
demo_train[demo_numeric_cols] = num_imputer.fit_transform(demo_train[demo_numeric_cols])
demo_test[demo_numeric_cols] = num_imputer.transform(demo_test[demo_numeric_cols])

# 非数值型列填充为最频繁值
cat_imputer = SimpleImputer(strategy='most_frequent')
cgm_train[cgm_non_numeric_cols] = cat_imputer.fit_transform(cgm_train[cgm_non_numeric_cols])
cgm_test[cgm_non_numeric_cols] = cat_imputer.transform(cgm_test[cgm_non_numeric_cols])
demo_train[demo_non_numeric_cols] = cat_imputer.fit_transform(demo_train[demo_non_numeric_cols])
demo_test[demo_non_numeric_cols] = cat_imputer.transform(demo_test[demo_non_numeric_cols])

# Step 4: 特征归一化
scaler = StandardScaler()
cgm_train_scaled = pd.DataFrame(scaler.fit_transform(cgm_train[cgm_numeric_cols]), columns=cgm_numeric_cols)
cgm_test_scaled = pd.DataFrame(scaler.transform(cgm_test[cgm_numeric_cols]), columns=cgm_numeric_cols)

# Step 5: 类别型特征独热编码
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
demo_train_encoded = pd.DataFrame(encoder.fit_transform(demo_train[demo_non_numeric_cols]))
demo_test_encoded = pd.DataFrame(encoder.transform(demo_test[demo_non_numeric_cols]))

# Step 6: 数据合并
train_data = pd.concat([cgm_train_scaled, demo_train_encoded, img_train], axis=1)
test_data = pd.concat([cgm_test_scaled, demo_test_encoded, img_test], axis=1)

# Step 7: 添加标签
label_column_name = 'Breakfast Calories'  # 使用正确的标签列名
if label_column_name in label_train.columns:
    train_data['label'] = label_train[label_column_name]
else:
    raise KeyError(f"标签列 '{label_column_name}' 不存在于 label_train 数据框中")

# Step 8: 输出检查
print("训练数据样本：\n", train_data.head())
print("测试数据样本：\n", test_data.head())

# Step 9: 保存预处理后的数据
train_data.to_csv('preprocessed_train_data.csv', index=False)
test_data.to_csv('preprocessed_test_data.csv', index=False)

print("数据预处理完成并保存！")

训练数据样本：
    Subject ID       Day  Breakfast Time  Lunch Time    0    1    2    3    4   
0    -1.53819 -1.549193        0.284720    0.271934  0.0  1.0  0.0  0.0  0.0  \
1    -1.53819 -1.161895        0.284750    0.271968  0.0  1.0  0.0  0.0  0.0   
2    -1.53819 -0.774597        0.284779    0.271996  0.0  1.0  0.0  0.0  0.0   
3    -1.53819 -0.387298        0.284809    0.272027  0.0  1.0  0.0  0.0  0.0   
4    -1.53819  0.000000        0.284837    0.272057  0.0  0.0  1.0  0.0  0.0   

     5  ...   35   36   37  Subject ID  Day  Breakfast Fiber  Lunch Fiber   
0  0.0  ...  0.0  0.0  0.0           1    2              0.0           10  \
1  0.0  ...  0.0  0.0  0.0           1    3              0.0            4   
2  0.0  ...  0.0  0.0  0.0           1    4              0.0            5   
3  0.0  ...  0.0  0.0  0.0           1    5              7.0            5   
4  0.0  ...  0.0  0.0  0.0           1    6              0.0           18   

                              Image Before Brea